# Model Testing
> Final testing results for best models

In [1]:
%load_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
#import time # !
import json
#from matplotlib import pyplot as plt

from loadutils import retrieve_decoder_model, retrieve_model
import evaluation_helper as eh
import tensorflow as tf
import pandas as pd
from keras.utils import to_categorical

import glove_helper
from loadutils import conll2003Data, saveProcessedData, retrieve_model
from common import vocabulary, utils


/home/dhuber/miniconda3/envs/capsnet/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
TRAIN_FILE = "../data/CoNLL-2003_NeuroNER/en/train.txt"
DEV_FILE = "../data/CoNLL-2003_NeuroNER/en/valid.txt"
TEST_FILE = "../data/CoNLL-2003_NeuroNER/en/test.txt"

## Local helper utils

In [3]:
def construct_embedding_matrix(embed_dim, vocab_size):
    """
    construct embedding matrix from GloVe 6Bn word data
    
    reuse glove_helper code from w266 
    
    Returns: an embedding matrix directly plugged into keras.layers.Embedding(weights=[embedding_matrix])
    """
    reload(glove_helper)
    hands = glove_helper.Hands(ndim=embed_dim)
    embedding_matrix = np.zeros((vocab_size, embed_dim))
    
    for i in range(vocabData.vocab.size):
        word = vocabData.vocab.ids_to_words([i])[0]
        try:
            embedding_vector = hands.get_vector(word)
        except:
            embedding_vector = hands.get_vector("<unk>")
        embedding_matrix[i] = embedding_vector

    return embedding_matrix

## Load the Data

In [4]:
# UPDATES!
global_max_features = 20000
windowLength = 11
#testNumSents = 20000

# Use training set to build vocab here
vocabData = conll2003Data(TRAIN_FILE)
vocabData.buildVocab( vocabSize=global_max_features)

# Format training data
trainX, trainX_pos, trainX_capitals, trainY  = vocabData.formatWindowedData( 
                                                  vocabData.train_sentences, 
                                                  windowLength=windowLength,
                                                  verbose=False)

# read in dev data
devSents = vocabData.readFile( DEV_FILE)
devX, devX_pos, devX_capitals, devY = vocabData.formatWindowedData( 
                                              devSents, 
                                              windowLength=windowLength,
                                              verbose=False)

# read in the test data
testSents = vocabData.readFile( TEST_FILE)
testX, testX_pos, testX_capitals, testY = vocabData.formatWindowedData( 
                                                testSents, 
                                                windowLength=windowLength,
                                                verbose=False)

----------------------------------------------------
reading file from path ../data/CoNLL-2003_NeuroNER/en/train.txt
'readFile'  1195.10 ms
----------------------------------------------------
building vocabulary from TRAINING data...
'buildVocab'  1045.47 ms
----------------------------------------------------
formatting sentences into input windows...
'formatWindowedData'  1972.02 ms
----------------------------------------------------
reading file from path ../data/CoNLL-2003_NeuroNER/en/valid.txt
'readFile'  254.26 ms
----------------------------------------------------
formatting sentences into input windows...
'formatWindowedData'  454.07 ms
----------------------------------------------------
reading file from path ../data/CoNLL-2003_NeuroNER/en/test.txt
'readFile'  232.78 ms
----------------------------------------------------
formatting sentences into input windows...
'formatWindowedData'  440.10 ms


In [5]:
# Load GloVe embedding matrix

global_embed_dim = 50

embedding_matrix = construct_embedding_matrix( global_embed_dim, 
                                               global_max_features)

Loading vectors from data/glove/glove.6B.zip
Parsing file: data/glove/glove.6B.zip:glove.6B.50d.txt
Found 400,000 words.
Parsing vectors... Done! (W.shape = (400003, 50))


In [6]:
# Get Y

# encoding 1-hot for ner targets
trainY_cat = to_categorical(trainY.astype('float32'))
devY_cat = to_categorical(devY.astype('float32'), num_classes=trainY_cat.shape[1])
testY_cat = to_categorical(testY.astype('float32'), num_classes=trainY_cat.shape[1])

trainY_cat = np.array(list(map( lambda i: np.array(i[3:], dtype=np.float), trainY_cat)), dtype=np.float)
devY_cat = np.array(list(map( lambda i: np.array(i[3:], dtype=np.float), devY_cat)), dtype=np.float)
testY_cat = np.array(list(map( lambda i: np.array(i[3:], dtype=np.float), testY_cat)), dtype=np.float)

In [7]:
# Get decoder Y -- 50 dim embedding of center word
train_decoderY = embedding_matrix[trainX[:,4]]
dev_decoderY = embedding_matrix[devX[:,4]]
test_decoderY = embedding_matrix[testX[:,4]]

In [8]:
# Get X pos tags

# encoding 1-hot for pos tags
trainX_pos_cat = to_categorical(trainX_pos.astype('float32'))
devX_pos_cat = to_categorical(devX_pos.astype('float32'), num_classes=trainX_pos_cat.shape[2]) 
testX_pos_cat = to_categorical(testX_pos.astype('float32'), num_classes=trainX_pos_cat.shape[2])

trainX_pos_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), trainX_pos_cat)), dtype=np.float)
devX_pos_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), devX_pos_cat)), dtype=np.float)
testX_pos_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), testX_pos_cat)), dtype=np.float)

In [9]:
# Get X capitlization 

# encoding 1-hot for capitalization info  ("allCaps", "upperInitial", "lowercase", "mixedCaps", "noinfo")
trainX_capitals_cat = to_categorical(trainX_capitals.astype('float32'))
devX_capitals_cat = to_categorical(devX_capitals.astype('float32'), num_classes=trainX_capitals_cat.shape[2]) 
testX_capitals_cat = to_categorical(testX_capitals.astype('float32'), num_classes=trainX_capitals_cat.shape[2])

trainX_capitals_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), trainX_capitals_cat)), dtype=np.float)
devX_capitals_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), devX_capitals_cat)), dtype=np.float)
testX_capitals_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), testX_capitals_cat)), dtype=np.float)

## Final Models

In [10]:
devX_dict = {"x":devX, "x_pos":devX_pos_cat, "x_capital":devX_capitals_cat}
testX_dict = {"x":testX, "x_pos":testX_pos_cat, "x_capital":testX_capitals_cat}
modelDir = { "save_dir" : "result"}

### Model A

In [11]:
# model A - Window11_glove50_learn_drop50_caps3_conv5_cos_pos_caps
modelName = "Window11_glove50_learn_drop50_caps3_conv5_cos_pos_caps"
myModel = retrieve_model( modelName, modelDir)
myModel.summary()

raw_y_pred = myModel.predict( testX_dict)
y_pred = eh.convert_raw_y_pred( raw_y_pred)

# construct report object
report_obj4 = eh.EvalDev_Report(modelName=modelName, y_true=testY, raw_y_pred=raw_y_pred, y_pred=y_pred) 
report_obj4.connect_to_dataClass( vocabData)

print("Precision", report_obj4.get_precision( testY, y_pred))
print("Recall\t", report_obj4.get_recall( testY, y_pred))
print("F1\t", report_obj4.get_f1( testY, y_pred))
cm = report_obj4.get_confusion_matrix( testY, y_pred)
print()
cm

Retrieving model: Window11_glove50_learn_drop50_caps3_conv5_cos_pos_caps
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x (InputLayer)                  (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 11, 50)       1000000     x[0][0]                          
__________________________________________________________________________________________________
x_pos (InputLayer)              (None, 11, 45)       0                                            
__________________________________________________________________________________________________
x_capital (InputLayer)          (None, 11, 5)        0                                            
____________________________________

,O,B-LOC,B-PER,B-ORG,I-PER,I-ORG,B-MISC,I-LOC,I-MISC
O,38027,33,53,38,7,36,71,15,43
B-LOC,14,1543,37,42,1,2,23,4,2
B-PER,29,23,1529,21,6,1,6,1,1
B-ORG,76,96,103,1333,0,13,40,0,0
I-PER,25,0,10,1,1096,23,0,1,0
I-ORG,61,5,4,23,18,665,3,43,13
B-MISC,62,24,30,30,0,0,550,0,6
I-LOC,14,6,1,0,3,6,0,223,4
I-MISC,39,0,2,1,3,15,8,3,145


### Model B (Model A + Decoder)

In [12]:
# model B - Window11_glove50_learn_drop50_decode1000_caps3_conv5_cos_pos_caps
modelName = "Window11_glove50_learn_drop50_decode1000_caps3_conv5_cos_pos_caps"
myModel = retrieve_decoder_model( modelName, modelDir)
myModel.summary()

raw_y_pred, raw_eval_y_pred = myModel.predict( testX_dict)
y_pred = eh.convert_raw_y_pred( raw_y_pred)

# construct report object
report_obj4 = eh.EvalDev_Report(modelName=modelName, y_true=testY, raw_y_pred=raw_y_pred, y_pred=y_pred) 
report_obj4.connect_to_dataClass( vocabData)

print("Precision", report_obj4.get_precision( testY, y_pred))
print("Recall\t", report_obj4.get_recall( testY, y_pred))
print("F1\t", report_obj4.get_f1( testY, y_pred))
cm = report_obj4.get_confusion_matrix( testY, y_pred)
print()
cm

Retrieving model_eval: Window11_glove50_learn_drop50_decode1000_caps3_conv5_cos_pos_caps
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x (InputLayer)                  (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 11, 50)       1000000     x[0][0]                          
__________________________________________________________________________________________________
x_pos (InputLayer)              (None, 11, 45)       0                                            
__________________________________________________________________________________________________
x_capital (InputLayer)          (None, 11, 5)        0                                            
____________________

,O,B-LOC,B-PER,B-ORG,I-PER,I-ORG,B-MISC,I-LOC,I-MISC
O,37981,35,18,104,8,39,79,9,50
B-LOC,12,1534,10,73,1,1,33,1,3
B-PER,26,27,1418,118,18,1,6,1,2
B-ORG,47,89,42,1424,0,16,41,0,2
I-PER,11,2,3,3,1110,21,0,4,2
I-ORG,55,13,1,18,19,669,4,34,22
B-MISC,56,12,10,63,0,0,554,0,7
I-LOC,18,9,0,0,5,6,0,217,2
I-MISC,45,1,2,3,2,8,7,3,145


### CNN

In [13]:
# CNN  -- Win11_cnn_glove50_learn_drop50_convtwo5_convone3_cos_pos_caps
modelName = "Win11_cnn_glove50_learn_drop50_convtwo5_convone3_cos_pos_caps"
myModel = retrieve_model( modelName, modelDir)
myModel.summary()

raw_y_pred = myModel.predict( testX_dict)
y_pred = eh.convert_raw_y_pred( raw_y_pred)

# construct report object
report_obj4 = eh.EvalDev_Report(modelName=modelName, y_true=testY, raw_y_pred=raw_y_pred, y_pred=y_pred) 
report_obj4.connect_to_dataClass( vocabData)

print("Precision", report_obj4.get_precision( testY, y_pred))
print("Recall\t", report_obj4.get_recall( testY, y_pred))
print("F1\t", report_obj4.get_f1( testY, y_pred))
cm = report_obj4.get_confusion_matrix( testY, y_pred)
print()
cm

Retrieving model: Win11_cnn_glove50_learn_drop50_convtwo5_convone3_cos_pos_caps
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x (InputLayer)                  (None, 11)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 11, 50)       1000000     x[0][0]                          
__________________________________________________________________________________________________
x_pos (InputLayer)              (None, 11, 45)       0                                            
__________________________________________________________________________________________________
x_capital (InputLayer)          (None, 11, 5)        0                                            
_____________________________

,O,B-LOC,B-PER,B-ORG,I-PER,I-ORG,B-MISC,I-LOC,I-MISC
O,37944,31,59,78,19,78,49,12,53
B-LOC,17,1509,29,79,1,1,26,4,2
B-PER,59,25,1454,59,11,2,5,1,1
B-ORG,69,85,67,1395,0,26,18,0,1
I-PER,55,0,10,0,1064,26,0,0,1
I-ORG,44,1,2,23,19,702,2,34,8
B-MISC,66,22,29,57,0,1,520,0,7
I-LOC,10,3,0,0,14,20,1,206,3
I-MISC,31,0,2,2,7,16,8,5,145
